## Preprocess MNIST dataset (locally without Beam/Dataflow)

In [5]:
import tensorflow as tf
tf.__version__

'1.8.0'

## Set your bucket name

In [1]:
BUCKET_NAME=''


In [8]:
import tensorflow as tf
import os

try:
  ROOT_DIR = 'gs://{}'.format(BUCKET_NAME)
except NameError:
  ROOT_DIR = './tutorial'
  
DATA_DIR = '{}/mnist-data'.format(ROOT_DIR)

# Remove CHECKPOINT_DIR if needed
  
if not tf.gfile.IsDirectory(DATA_DIR):
  tf.logging.info('create {}'.format(DATA_DIR))
  tf.gfile.MkDir(ROOT_DIR)
  tf.gfile.MkDir(DATA_DIR)

## Select a subset of records

In [9]:
import numpy as np

train, test = tf.keras.datasets.mnist.load_data()
X_train = train[0][:-5000]
y_train = train[1][:-5000]
X_eval = train[0][-5000:]
y_eval = train[1][-5000:]
X_test = test[0]
y_test = test[1]

## Convert the dataset (image and labels) to tfrecords and store on Google Cloud Storage

In [10]:
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_example(image, label):  
  feature={
      'image': _bytes_feature(image.tobytes()),
      'label': _bytes_feature(label.tobytes())}
  return tf.train.Example(features=tf.train.Features(feature=feature))

def convert_to_tfrecord(images, labels, output_file):
  with tf.python_io.TFRecordWriter(output_file) as record_writer:
    for image, label in zip(images, labels):
      example = create_example(image, label)
      record_writer.write(example.SerializeToString())
      
      
convert_to_tfrecord(X_train, y_train,
                    output_file='{}/train.tfrecord'.format(DATA_DIR))
convert_to_tfrecord(X_eval, y_eval,
                    output_file='{}/eval.tfrecord'.format(DATA_DIR))
convert_to_tfrecord(X_test, y_test,
                    output_file='{}/test.tfrecord'.format(DATA_DIR))